# `Logit` on Orders

## Features Selection

🎯 The impact of `wait_time` and `delay_vs_expected` on very `good/bad reviews` is investigated

👉 Using the `orders` training_set, two `multivariate logistic regressions` are run:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

 

In [1]:
#Modules are imported
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

#Dataset is imported
from order import Order

%load_ext autoreload
%autoreload 2

In [2]:
orders = Order().get_training_data()

👉 The features to be used are selected in a list:

⚠️ Features that are derived from the target are not selected as these will create data leakage

💡 To figure out the impact of `wait_time` and `delay_vs_expected`, the impact of other features is controlled, all features that may be relevant are included

In [3]:
features_list = ["dim_is_one_star", "dim_is_five_star", "wait_time","delay_vs_expected", "price", "distance_seller_customer", "number_of_products", "number_of_sellers"]

🕵🏻 The `multi-colinearity` of the features are checked using the `VIF index`.

* It shouldn't be too high (< 10 preferably) to ensure that we can trust the partial regression coefficents and their associated `p-values` 
* Data must be standardized ! 
    * A `VIF Analysis` is made by regressing a feature vs. the other features...
    * So `removal of effect of scale` needs to be performed so that the features have an equal importance before running any linear regression!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Features are standardized:

In [4]:
orders_scaled = orders.copy()
orders_scaled = orders_scaled[features_list]

for feature in orders_scaled.columns[2:]:
    mu = orders_scaled[feature].mean()
    sigma = orders_scaled[feature].std()
    orders_scaled[feature] = orders_scaled[feature].apply(lambda x: (x-mu)/sigma)
    
orders_scaled.head()

,dim_is_one_star,dim_is_five_star,wait_time,delay_vs_expected,price,distance_seller_customer,number_of_products,number_of_sellers
0,0,0,-0.428002,-0.153335,-0.513802,-0.979475,-0.264595,-0.112544
1,0,0,0.100519,-0.153335,-0.086640,0.429743,-0.264595,-0.112544
2,0,1,-0.322297,-0.153335,0.111748,-0.145495,-0.264595,-0.112544
3,0,1,0.100519,-0.153335,-0.441525,2.054621,-0.264595,-0.112544
4,0,1,-1.062226,-0.153335,-0.562388,-0.959115,-0.264595,-0.112544


👉 VIF Analysis is run to analyze the potential multicolinearities:

In [5]:
vif_df = pd.DataFrame()

vif_df["features"] = orders_scaled.columns
vif_df["vif_index"] = [vif(orders_scaled.values, i) for i in range(orders_scaled.shape[1])]
round(vif_df.sort_values(by="vif_index", ascending = False),2)

vif_df

,features,vif_index
0,dim_is_one_star,1.133492
1,dim_is_five_star,1.031407
2,wait_time,2.648983
3,delay_vs_expected,2.151810
4,price,1.032502
5,distance_seller_customer,1.327266
6,number_of_products,1.128042
7,number_of_sellers,1.102739


## Logistic Regressions

👉 Two `Logistic Regression` models are fitted:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

`Logit 1️⃣`

In [6]:
logit_one = smf.logit(formula='''dim_is_one_star ~ wait_time + delay_vs_expected + price + 
distance_seller_customer + number_of_products + number_of_sellers''', data=orders_scaled).fit()

logit_one.summary()

Optimization terminated successfully.
         Current function value: 0.273940
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95865
Method:                           MLE   Df Model:                            6
Date:                Mon, 11 Jul 2022   Pseudo R-squ.:                  0.1436
Time:                        22:24:31   Log-Likelihood:                -26263.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -2.4710      0.013   -190.640      0.000      -2.496      -2.446
wait_time                    0.7448      0.017     45.101      0.000       0.712       0.777
delay_vs_expected            0.2124      0.018     11.606      0.000       0.177       0.248
price                        0.0425      0.011      4.051      0.000       0.022       0.063
distance_seller_customer    -0.1949      0.013    -14.608      0.000      -0.221      -0.169
number_of_products           0.2407      0.009     26.231      0.000       0.223       0.259
number_of_sellers            0.1785      0.008     22.710      0.000       0.163       0.194
============================================================================================
"""

`Logit 5️⃣`

In [7]:
logit_five = smf.logit(formula='''dim_is_five_star ~ wait_time + delay_vs_expected + price + 
distance_seller_customer + number_of_products + number_of_sellers''', data=orders_scaled).fit()

logit_five.summary()

Optimization terminated successfully.
         Current function value: 0.637179
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95865
Method:                           MLE   Df Model:                            6
Date:                Mon, 11 Jul 2022   Pseudo R-squ.:                 0.05754
Time:                        22:24:32   Log-Likelihood:                -61088.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.3406      0.007     47.577      0.000       0.327       0.355
wait_time                   -0.5349      0.012    -46.497      0.000      -0.557      -0.512
delay_vs_expected           -0.4114      0.024    -17.102      0.000      -0.459      -0.364
price                        0.0225      0.007      3.189      0.001       0.009       0.036
distance_seller_customer     0.0926      0.008     11.661      0.000       0.077       0.108
number_of_products          -0.1343      0.008    -17.727      0.000      -0.149      -0.119
number_of_sellers           -0.1428      0.008    -18.232      0.000      -0.158      -0.127
============================================================================================
"""

💡 the results of these two logistic regressions are analysed:

- The partial coefficients are interpreted.
- Their statistical significances with are checked `p-values`
- Are there any differences between `logit_one` and `logit_five` in terms of coefficient importance?

<details>
    <summary>- <i> Details </i> -</summary>


> _All other things being equal, the `delay factor` tends to increase the chances of getting stripped of the 5-star even more so than it affect the chances of 1-star reviews. Probably because 1-stars are really targeting bad products themselves, not bad deliveries_
    
❗️ However, to be totally rigorous, we have to be **more careful when comparing coefficients from two different models**, because **they might not be based on similar populations**!
    We have 2 sub-populations here: (people who gave 1-stars; and people who gave 5-stars) and they may exhibit intrinsically different behavior patterns. It may well be that "happy-people" (who tends to give 5-stars easily) are less sensitive as "grumpy-people" (who shoot 1-stars like Lucky-Luke), when it comes to "delay", or "price"...

</details>


## Logistic vs. Linear

👉 The regression coefficients obtained from the `Logistic Regression ` is compared with the regression coefficients obtained through a `Linear Regression` on `review_score`, using the same features. 

⚠️ Do both sets of coefficients  tell  "the same story"?

In [8]:
features = ["review_score", "wait_time","delay_vs_expected", "price", "distance_seller_customer", 
       "number_of_products", "number_of_sellers"]

orders_scaled_features = orders.copy()
orders_scaled_features
orders_scaled_features = orders_scaled_features[features]

for feature in orders_scaled_features.columns[1:]:
    mu = orders_scaled_features[feature].mean()
    sigma = orders_scaled_features[feature].std()
    orders_scaled_features[feature] = orders_scaled_features[feature].apply(lambda x: (x-mu)/sigma)
    
orders_scaled_features.head()

,review_score,wait_time,delay_vs_expected,price,distance_seller_customer,number_of_products,number_of_sellers
0,4,-0.428002,-0.153335,-0.513802,-0.979475,-0.264595,-0.112544
1,4,0.100519,-0.153335,-0.086640,0.429743,-0.264595,-0.112544
2,5,-0.322297,-0.153335,0.111748,-0.145495,-0.264595,-0.112544
3,5,0.100519,-0.153335,-0.441525,2.054621,-0.264595,-0.112544
4,5,-1.062226,-0.153335,-0.562388,-0.959115,-0.264595,-0.112544


In [9]:
ols_model = smf.ols(formula = '''review_score ~ wait_time + delay_vs_expected + price + 
distance_seller_customer + number_of_products + number_of_sellers''', data = orders_scaled_features)

ols_model = ols_model.fit()

ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           review_score   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     2698.
Date:                Mon, 11 Jul 2022   Prob (F-statistic):               0.00
Time:                        22:24:32   Log-Likelihood:            -1.5261e+05
No. Observations:               95872   AIC:                         3.052e+05
Df Residuals:                   95865   BIC:                         3.053e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    4.1555      0.004   1082.409      0.000       4.148       4.163
wait_time                   -0.4534      0.006    -74.213      0.000      -0.465      -0.441
delay_vs_expected           -0.0344      0.006     -6.117      0.000      -0.045      -0.023
price                       -0.0014      0.004     -0.368      0.713      -0.009       0.006
distance_seller_customer     0.1021      0.004     23.129      0.000       0.093       0.111
number_of_products          -0.1280      0.004    -31.546      0.000      -0.136      -0.120
number_of_sellers           -0.1317      0.004    -32.800      0.000      -0.140      -0.124
==============================================================================
Omnibus:                    18653.258   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            36496.248
Skew:                          -1.186   Prob(JB):                         0.00
Kurtosis:                       4.874   Cond. No.                         2.84
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""